<a href="https://colab.research.google.com/github/JasonkayZK/high-performance-computing-learn/blob/main/cuda/0-an-even-easier-intro-to-cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **一个简单的CUDA介绍**

> **参考文章：**
>
> - https://developer.nvidia.com/blog/even-easier-introduction-cuda/
> - https://colab.research.google.com/github/NVDLI/notebooks/blob/master/even-easier-cuda/An_Even_Easier_Introduction_to_CUDA.ipynb
> - https://beta.infinitensor.com/camp/summer2025/stage/1/course/cuda-programming?tab=1
> - https://qiankunli.github.io/2025/03/22/cuda.html


CUDA 是 NVIDIA 流行的并行计算平台和编程模型。

CUDA C++是使用 CUDA 创建大规模并行应用程序的方法之一。它允许你使用 C++ 编程语言来开发高性能算法，这些算法由在 GPU 上运行的数千个并行线程加速。

## **一、从一个数组加法开始**

我们将从一个简单的 C++ 程序开始，该程序添加两个数组的元素，每个数组有一百万个元素。


In [1]:
%%writefile add.cpp

#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y) {
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void) {
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host(CPU)
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}

Overwriting add.cpp


> `%%writefile` 命令用于写入文件；
>
> 执行上述单元格会将其内容保存到文件 add.cpp。

In [2]:
%%shell
# 编译
g++ add.cpp -o add

# 运行
./add

Max error: 0


正如预期的那样，上面的代码打印求和中没有错误，然后退出。

现在让这个计算在 GPU 的许多内核上（并行）运行。

首先，只需要将我们的 add 函数变成 GPU 可以运行的函数，在 CUDA 中称为内核(kernel)。

为此所要做的就是：将说明符 `__global__` 添加到函数中：**它告诉 CUDA C++编译器这是一个在 GPU 上运行的函数，可以从 CPU 代码调用：**

```c
// CUDA Kernel function to add the elements of two arrays on the GPU
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}
```

这些 `__global__` 函数称为内核(kernel)，在 GPU 上运行的代码通常称为设备代码(device code)，而在 CPU 上运行的代码是主机代码(host code)。

## **二、CUDA 中的内存分配**

要在 GPU 上进行计算，需要分配 GPU 可访问的内存。

CUDA 中的**[统一内存](https://developer.nvidia.com/blog/unified-memory-in-cuda-6/)(Unified Memory)**通过提供系统中所有 GPU 和 CPU 可访问的单个内存空间来实现内存分配。

要在统一内存中分配数据，需要调用 `cudaMallocManaged()`， 它返回一个指针；

可以从主机（CPU）代码或设备（GPU）代码访问该指针。

要释放数据，只需将指针传递给 `cudaFree()` 即可。

只需要将上面代码中对 new 的调用替换为对 `cudaMallocManaged()` 的调用，并将对 `delete []` 的调用替换为对 `cudaFree()` 的调用！

```c
  // Allocate Unified Memory -- accessible from CPU or GPU
  float *x, *y;
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  ...

  // Free memory
  cudaFree(x);
  cudaFree(y);
```

最后，需要启动 `add()` 内核，在 GPU 上调用它；

CUDA 内核启动使用：**三重尖括号语法** `<<< >>>` 指定。

只需要将其添加到参数列表之前的调用中即可：

```c
add<<<1, 1>>>(N, x, y);
```

> 后续会详细介绍尖括号内的内容；
>
> 现在你需要知道的是：这一行启动了一个 GPU 线程来运行 `add()`

还有一个问题：需要 CPU 等到内核函数 add 完成后再访问结果（**因为 CUDA 内核启动不会阻止调用的 CPU 线程**）！

为此，只需在对 CPU 进行最终错误检查之前调用 `cudaDeviceSynchronize()`！

下面是完整的代码：

In [3]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20
 ;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting add.cu


In [4]:
%%shell

nvcc add.cu -o add_cuda
./add_cuda

Max error: 1


通过 `nvcc` 可以编译 CUDA 程序；

而在编写代码时需要将 `*.cpp` 文件类型修改为 `*.cu` 类型表示是一个 CUDA 程序！

## **三、性能测试**

找出内核函数运行需要多长时间的最简单方法是使用：`nvprof` 运行：

nvprof 是 CUDA Toolkit 附带的命令行 GPU 分析器，只需在命令行上键入：

`nvprof ./add_cuda`

In [5]:
%%shell

nvprof ./add_cuda

==19628== NVPROF is profiling process 19628, command: ./add_cuda
Max error: 1
==19628== Profiling application: ./add_cuda
==19628== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   92.19%  92.918ms         2  46.459ms  48.457us  92.869ms  cudaMallocManaged
                    7.39%  7.4475ms         1  7.4475ms  7.4475ms  7.4475ms  cudaLaunchKernel
                    0.27%  269.24us         2  134.62us  118.45us  150.79us  cudaFree
                    0.12%  125.53us       114  1.1010us     105ns  51.627us  cuDeviceGetAttribute
                    0.01%  10.498us         1  10.498us  10.498us  10.498us  cuDeviceGetName
                    0.01%  8.1820us         1  8.1820us  8.1820us  8.1820us  cudaDeviceSynchronize
                    0.01%  5.1080us         1  5.1080us  5.1080us  5.1080us  cuDeviceGetPCIBusId
                    0.00%  1.4580us         3     486ns     115ns  1.1100us  cuD

> 若要查看 Colab 分配给你的当前 GPU，可以运行 `nvidia-smi`，然后查看 Name 列
>
> 例如，你可能会看到 Tesla T4：

In [6]:
%%shell

nvidia-smi

Fri Jul 25 08:09:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             27W /   70W |       0MiB /  15360MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

下面，让我们通过提升并行度让代码更快运行！

## **四、提高Threads**

在刚才的实验中，我们已经运行了一个带有一个执行一些计算的线程的内核。那么如何让它并行呢？

关键在于 CUDA 的 `<<<1、1>>>` 语法：这称为执行配置，它告诉 CUDA 运行时在 GPU 上启动时使用多少并行线程。

这里有两个参数，先让我们从更改第二个参数开始：**线程块中的线程数**。

**CUDA GPU 使用大小为 32 的倍数的线程块运行内核，因此 256 线程是一个合理的大小选择。**

```c
add<<<1, 256>>>(N, x, y);
```

但是如果只运行修改此处的代码，它将：**为每个线程都执行完整的循环计算，而不是将循环分散到并行线程中！**

> **这是由于 GPU 是 SIMT 的执行方式！**

为了正确地做到这一点，需要修改内核：CUDA C++ 提供了让内核获取正在运行的线程的索引的关键字。

具体来说，threadIdx.x 包含其块内当前线程的索引，blockDim.x 包含块中的线程数；

因此需要修改循环以使用并行线程在数组中以 stride 的长度前进；

```c
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}
```

add 函数没有太大变化，只是：**将循环的任务分配到了不同的 thread 中！**

> 事实上，将 index 设置为 0 并将 stride 设置为 1 在语义上与第一个版本相同！

将文件保存为 add_block.cu，并在 nvprof 中再次编译和运行：

In [7]:
%%writefile add_block.cu

#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 256>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting add_block.cu


In [8]:
%%shell

nvcc add_block.cu -o add_block
nvprof ./add_block

==19696== NVPROF is profiling process 19696, command: ./add_block
Max error: 1
==19696== Profiling application: ./add_block
==19696== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   93.94%  122.13ms         2  61.064ms  58.832us  122.07ms  cudaMallocManaged
                    5.72%  7.4305ms         1  7.4305ms  7.4305ms  7.4305ms  cudaLaunchKernel
                    0.22%  279.79us         2  139.90us  122.77us  157.02us  cudaFree
                    0.10%  136.08us       114  1.1930us     105ns  57.117us  cuDeviceGetAttribute
                    0.01%  11.251us         1  11.251us  11.251us  11.251us  cuDeviceGetName
                    0.01%  10.418us         1  10.418us  10.418us  10.418us  cudaDeviceSynchronize
                    0.01%  8.0560us         1  8.0560us  8.0560us  8.0560us  cuDeviceGetPCIBusId
                    0.00%  1.6270us         3     542ns     121ns  1.2840us  c

通过查看 GPU 活动字段来比较添加内核的时间：可以看到这是一个很大的加速！

让我们继续获得更多的性能！

## **五、多个Block**

CUDA GPU 都具有许多并行处理器，这些处理器被称为流式多处理器(SM, Streaming Multiprocessor)。每个 SM 可以运行多个并发线程块。

例如，基于 [Pascal GPU 架构](https://developer.nvidia.com/blog/inside-pascal/)的 Tesla P100 GPU 有 56 个 SM，每个 SM 能够支持多达 2048 个活动线程。

为了充分利用所有这些线程，应该启动具有多个线程块的内核！

> **类似于每个CPU核中都可以执行2048个线程！**

而执行配置中的：**第一个参数指定了线程块的数量！**这些并行线程块共同构成了所谓的网格。

对于 N 个元素要处理，每个块有 256 个线程，只需要计算块数即可获得 N 个线程。

将 N 除以块大小（如果 N 不是 blockSize 的倍数，需要小心四舍五入！）

```c
int blockSize = 256;
int numBlocks = (N + blockSize - 1) / blockSize;
add<<<numBlocks, blockSize>>>(N, x, y);
```

![cuda_indexing](https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_indexing.png)

> 上图说明了使用 blockDim.x、gridDim.x 和 threadIdx.x 在 CUDA 中索引到数组（一维）的方法！

还需要更新内核代码以考虑线程块所在的网格：

CUDA 提供了 gridDim.x，其中包含网格中的块数，以及 blockIdx.x，参数包含了网格中当前线程块的索引；

思路是：

- 每个线程通过计算到其块开头的偏移量（块索引乘以块大小：blockIdx.x * blockDim.x）
- 并在块内添加线程的索引（threadIdx.x）来获取其索引

这里的代码：`blockIdx.x * blockDim.x + threadIdx.x` 是惯用的技巧！

```c
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}
```

更新后的内核还设置了网格中的线程总数 （blockDim.x * gridDim.x）；

CUDA 内核中这种类型的循环通常称为[网格步幅循环(grid-stride)](https://developer.nvidia.com/blog/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/)。

下面，将文件另存为 add_grid.cu，然后再次在 nvprof 中编译和运行它。

In [9]:
%%writefile add_grid.cu

#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting add_grid.cu


In [10]:
%%shell

nvcc add_grid.cu -o add_grid
nvprof ./add_grid

==19751== NVPROF is profiling process 19751, command: ./add_grid
Max error: 1
==19751== Profiling application: ./add_grid
==19751== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   93.35%  113.28ms         2  56.639ms  44.803us  113.23ms  cudaMallocManaged
                    6.20%  7.5263ms         1  7.5263ms  7.5263ms  7.5263ms  cudaLaunchKernel
                    0.26%  315.41us         2  157.71us  130.85us  184.56us  cudaFree
                    0.15%  186.99us       114  1.6400us     142ns  77.782us  cuDeviceGetAttribute
                    0.02%  21.752us         1  21.752us  21.752us  21.752us  cuDeviceGetName
                    0.01%  8.4250us         1  8.4250us  8.4250us  8.4250us  cudaDeviceSynchronize
                    0.01%  6.9510us         1  6.9510us  6.9510us  6.9510us  cuDeviceGetPCIBusId
                    0.00%  2.0900us         3     696ns     218ns  1.5980us  cuD

这是运行多个块的巨大加速！

## **六、后续**

### **1、提升内容**

- 在内核中试验`printf()`。 尝试打印出部分或全部线程的 threadIdx.x 和 blockIdx.x 的值。他们是按顺序打印的吗？为什么或为什么不？
- 在内核中打印 threadIdx.y 或 threadIdx.z（或 blockIdx.y）的值。（blockDim 和 gridDim 也是如此）。为什么存在这些？如何让它们采用 0 以外的值（1 表示暗淡）？


### **2、后续阅读**

-   [How to Implement Performance Metrics in CUDA C++](https://developer.nvidia.com/blog/how-implement-performance-metrics-cuda-cc/)
-   [How to Query Device Properties and Handle Errors in CUDA C++](https://developer.nvidia.com/blog/how-query-device-properties-and-handle-errors-cuda-cc/)
-   [How to Optimize Data Transfers in CUDA C++](https://developer.nvidia.com/blog/how-optimize-data-transfers-cuda-cc/)
-   [How to Overlap Data Transfers in CUDA C++](https://developer.nvidia.com/blog/how-overlap-data-transfers-cuda-cc/)
-   [How to Access Global Memory Efficiently in CUDA C++](https://developer.nvidia.com/blog/how-access-global-memory-efficiently-cuda-c-kernels/)
-   [Using Shared Memory in CUDA C++](https://developer.nvidia.com/blog/using-shared-memory-cuda-cc/)
-   [An Efficient Matrix Transpose in CUDA C++](https://developer.nvidia.com/blog/efficient-matrix-transpose-cuda-cc/)
-   [Finite Difference Methods in CUDA C++, Part 1](https://developer.nvidia.com/blog/finite-difference-methods-cuda-cc-part-1/)
-   [Finite Difference Methods in CUDA C++, Part 2](https://developer.nvidia.com/blog/finite-difference-methods-cuda-c-part-2/)
-   [Accelerated Ray Tracing in One Weekend with CUDA](https://developer.nvidia.com/blog/accelerated-ray-tracing-cuda/)

对于刚接触CUDA的同学：

- [_Fundamentals of Accelerated Computing with CUDA C/C++_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-01+V1/about)：提供了专用的 GPU 资源、更复杂的编程环境。
- [NVIDIA Nsight Systems™](https://developer.nvidia.com/nsight-systems)：可视化分析器的使用、数十个交互式练习、详细的演示、超过 8 小时的材料，以及获得 DLI 能力证书的能力。
- [_Fundamentals of Accelerated Computing with CUDA Python_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-02+V1/about)：Python 程序员阅读

